In [6]:
import os.path as op
from glob import glob

from neuromaps.datasets import fetch_atlas, fetch_fslr
from netneurotools import stats as nnstats
from surfplot.utils import add_fslr_medial_wall
import numpy as np
import nibabel as nib

In [3]:
data_dir = op.abspath("../data")
result_dir = op.abspath("../results")
neuromaps_dir = op.join(data_dir, "neuromaps-data")

In [ ]:
from surfplot import Plot
from surfplot.utils import threshold

surfaces = fetch_fslr(density="32k", data_dir=neuromaps_dir)
lh, rh = surfaces["inflated"]
sulc_lh, sulc_rh = surfaces["sulc"]

def plot_surf_map(lh_grad, rh_grad, threshold_=None, cmap=None, color_range=None):
    if threshold_ is not None:
        lh_grad = threshold(lh_grad, threshold_)
        rh_grad = threshold(rh_grad, threshold_)

    p = Plot(surf_lh=lh, surf_rh=rh)
    p.add_layer({"left": sulc_lh, "right": sulc_rh}, cmap="binary_r", cbar=False)
    p.add_layer({"left": lh_grad, "right": rh_grad}, cmap=cmap, color_range=color_range)

    fig = p.build()

In [12]:

atlas = fetch_atlas("fsLR", "32k", data_dir=neuromaps_dir, verbose=0)

sphere_lh, sphere_rh = atlas["sphere"]
coords_lh, _ = nib.load(sphere_lh).agg_data()
coords_rh, _ = nib.load(sphere_rh).agg_data()

medial_lh, medial_rh = atlas["medial"]
medial_arr_lh = nib.load(medial_lh).agg_data()
medial_arr_rh = nib.load(medial_rh).agg_data()

coords_lh = coords_lh[np.where(medial_arr_lh != 0)]
coords_rh = coords_rh[np.where(medial_arr_rh != 0)]

hemi = np.hstack((np.zeros((coords_lh.shape[0], )), np.ones((coords_rh.shape[0], ))))
coords = np.vstack((coords_lh, coords_rh))

In [16]:
permute = nnstats.gen_spinsamples(coords, hemi, n_rotate=1, seed=1, method='vasa', check_duplicates=False)

In [15]:
gclda_grad_path = op.join(result_dir, "gradient_decoding", "gclda-fslr")

gclda_grad_lh_fnames = sorted(glob(op.join(gclda_grad_path, "*hemi-L_feature.func.gii")))[0]
gclda_grad_rh_fnames = sorted(glob(op.join(gclda_grad_path, "*hemi-R_feature.func.gii")))[0]
lh_grad = nib.load(gclda_grad_lh_fnames).agg_data()
rh_grad = nib.load(gclda_grad_rh_fnames).agg_data()

plot_surf_map(lh_grad, lh_grad)

(59412,)

In [ ]:
lh_grad_permuted = lh_grad[np.where(medial_arr_lh != 0)]
rh_grad = rh_grad[np.where(medial_arr_rh != 0)]
grad_map = np.hstack((lh_grad, rh_grad))

grad_map_permuted = grad_map[permute]

lh_grad_permuted, rh_grad_permuted = add_fslr_medial_wall(grad_map_permuted, split=True)

plot_surf_map(lh_grad_permuted, rh_grad_permuted)